#### 1 read correct
##### Quake下载和安装
```bash
wget http://www.cbcb.umd.edu/software/quake/downloads/quake-0.3.5.tar.gz
tar -vxf quake-0.3.5.tar.gz 
cd Quake/src/
make
cd ../bin/
./correct -h
```
##### Quake运行           Fail
```bash
python2 ~/software/Quake/bin/quake.py  -f filenames.tab -k 19 # filenames.tab保存fastq格式的原始文件路径，配对文件用空格格开
./correct
```

## 1 原始数据预处理

### 1.1 原始数据质量控制
```bash
java -jar /home/wxu/software/Trimmomatic-0.36/trimmomatic-0.36.jar PE -phred33 -threads 24 "../data/PMSuperPoolB_Y_R1.fastq.gz" "../data/PMSuperPoolB_Y_R2.fastq.gz" "paired_PMSuperPoolB_Y_R1.fastq.gz" "unpaired_PMSuperPoolB_Y_R1.fastq.gz" "paired_PMSuperPoolB_Y_R2.fastq.gz" "unpaired_PMSuperPoolB_Y_R2.fastq.gz" HEADCROP:4 LEADING:20 TRAILING:20 ILLUMINACLIP:./adaptor.fasta:2:18:7:2:true SLIDINGWINDOW:5:20 AVGQUAL:23 MINLEN:70 


fastqc -t 24 paired_PMSuperPoolB_Y_R1.fastq.gz

fastqc -t 24 paired_PMSuperPoolB_Y_R2.fastq.gz


java -jar /home/wxu/software/Trimmomatic-0.36/trimmomatic-0.36.jar PE -phred33 -threads 24 "../data/PMSuperPoolB_X_R1.fastq.gz" "../data/PMSuperPoolB_X_R2.fastq.gz" "paired_PMSuperPoolB_X_R1.fastq.gz" "unpaired_PMSuperPoolB_X_R1.fastq.gz" "paired_PMSuperPoolB_X_R2.fastq.gz" "unpaired_PMSuperPoolB_X_R2.fastq.gz" HEADCROP:4 LEADING:20 TRAILING:20 ILLUMINACLIP:./adaptor.fasta:2:18:7:2:true SLIDINGWINDOW:5:20 AVGQUAL:23 MINLEN:70


fastqc -t 24 paired_PMSuperPoolB_X_R1.fastq.gz

fastqc -t 24 paired_PMSuperPoolB_X_R2.fastq.gz
```

### 1.2 污染序列去除（大肠杆菌和Phi174）
```bash
bowtie2 --threads 18 --un-conc ./CleanReads_PMSuperPoolB_X -x /home/wxu/data/genome/E_coli/ecoli_DH10B.fasta -1 ../trim/paired_PMSuperPoolB_X_R1.fastq.gz -2 ../trim/paired_PMSuperPoolB_X_R2.fastq.gz -S ./PM-B-pool-X-align.sam

bowtie2 --threads 18 --un-conc ./CleanReads_PMSuperPoolB_Y -x /home/wxu/data/genome/E_coli/ecoli_DH10B.fasta -1 ../trim/paired_PMSuperPoolB_Y_R1.fastq.gz -2 ../trim/paired_PMSuperPoolB_Y_R2.fastq.gz -S ./PM-B-pool-Y-align.sam

```

### 1.3 数据拆分
```bash
# index.id文件的每行是每个混合池建库过程中使用的index序列
~/software/bbmap/demuxbyname.sh in=/home/wxu/PMSuperPoolB/CleanReads/CleanReads_PMSuperPoolB_X.1 in2=/home/wxu/PMSuperPoolB/CleanReads/CleanReads_PMSuperPoolB_X.2 out=X_%_R1.fastq out2=X_%_R2.fastq names=index.id prefixmode=f

~/software/bbmap/demuxbyname.sh in=/home/wxu/PMSuperPoolB/CleanReads/CleanReads_PMSuperPoolB_Y.1 in2=/home/wxu/PMSuperPoolB/CleanReads/CleanReads_PMSuperPoolB_Y.2 out=Y_%_R1.fastq out2=Y_%_R2.fastq names=index.id prefixmode=f

# 输出的文件是以index序列命名的，需要用对应的混合池编号重命名
```


### 2 混合池末端序列解析
#### 2.1 短末端序列
```bash
# VES-BES reads提取
python /home/wxu/workspace/BES-simulation/RICE/BES/BES_F_optimize/filtered_reads_from_NGS_20181107.py -v /home/wxu/data/vector/pIndigoBAC536S_F.fa -1 /home/wxu/workspace/PM-B-XY/data/${index}_R1.fastq -2 /home/wxu/workspace/PM-B-XY/data/${index}_R2.fastq -a ${index}_filtered_R1.fastq -b ${index}_filtered_R2.fastq
# 文件格式转换和合并
python /home/wxu/software/toolkits/fastq2fasta.py -i ../1_filtered_NGS/${index}_filtered_R1.fastq -o ${index}_R1.fasta
python /home/wxu/software/toolkits/fastq2fasta.py -i ../1_filtered_NGS/${index}_filtered_R2.fastq -o ${index}_R2.fasta
cat ${index}_R1.fasta ${index}_R2.fasta > ${index}.fasta

rm ${index}_R1.fasta
rm ${index}_R2.fasta

# Assembled to Contig
cap3 ${index}.fasta > ${index}.log

rm ${index}.log
rm ${index}.fasta.cap.info
rm ${index}.fasta.cap.singlets
rm ${index}.fasta.cap.ace
rm ${index}.fasta.cap.contigs.qual
rm ${index}.fasta.cap.contigs.links



#scafolding with SCARPA
/home/wxu/software/velvet_1.2.10/contrib/shuffleSequences_fasta/shuffleSequences_fastq.pl ../1_filtered_NGS/${index}_filtered_R1.fastq ../1_filtered_NGS/${index}_filtered_R2.fastq ${index}_R12.fastq
/home/wxu/software/scarpa-0.241/bin/scarpa_process -c ${index}.fasta.cap.contigs -f ${index}_R12.fastq -i 500
bowtie2-build ${index}.fasta.cap.contigs.scarpa.fa ${index}_index
bowtie2 -x ${index}_index -U ${index}_R12.fastq.scarpa.fq | /home/wxu/software/scarpa-0.241/bin/scarpa_parser > ${index}_sam.scarpa
/home/wxu/software/scarpa-0.241/bin/scarpa -c  ${index}.fasta.cap.contigs.scarpa.fa -i ${index}_sam.scarpa -l  ${index}.fasta.cap.contigs.scarpa.info -o ${index}.scaf.fasta

rm ${index}*.bt2



# Trimming Vector End Sequence
blat /home/wxu/data/vector/pIndigoBAC536S_F.fa ${index}.scaf.fasta -noHead ${index}.psl
python /home/wxu/software/toolkits/trim_vector_from_BES_by_blat_20181108.py -f ${index}.scaf.fasta -p ${index}.psl -o ${index}_BES.fasta
```

#### 2.2 长末端序列

组装每个混合池序列

```bash
# index为混合池编号
python ~/software/SPAdes-3.13.0-Linux/bin/spades.py -t 7 -k 31,51,71,91 --pe1-1 ../data/${index}_R1.fastq --pe1-2 ../data/${index}_R2.fastq -o ${index}

cp ./${index}/contigs.fa ${index}_contigs.fa

~/software/mashmap-Linux64-v2.0/mashmap -r ~/data/vector/BAC-contamination.fa -q ${index}/contigs.fasta --pi 95 -o ${index}/mashmap.out

python decontainment_from_contigs_by_mashmap.20190219.py -p ${index}/mashmap.out -f ${index}/contigs.fasta -o ${index}_long_contigs.fa
```

从混合池组装结果中提取长末端序列

~/software/toolkits/extract_aligned_sequence_20190314.py

#### 2.3 末端序列注释
```bash
/home/hwang/biosoftware/RepeatMasker/RepeatMasker -parallel 20 -species maize -html -gff -dir long_BES_mask long_BES.fasta
```

### 3 BAC末端序列交叉定孔

```Python3
import numpy as np
from gresource import plate
from gresource.align import TwoScaffoldsAlignment
from gresourceold.plate import  flatten
import HTSeq

#layout_id = plate.parse_pool_identity(np.array([['1','2'],['3','4'],['5','6']]))
layout_id = plate.superpool_2 #糜子384板的布局（25-48）


from HTSeq import FastaReader
import os


handle_file_valid = open('BES_valid.fasta','w') #位点仅单条序列
handle_file_invalid = open('BES_invalid.fasta', 'w') #位点有多条序列的情况
handle_file = open('BES_ALL.fasta','w+')

BES_library_0 = {} # initial dict

num_bes = np.zeros((96,96), dtype='int8')  #建立空的矩阵，用于存储每个位点解析出的末端序列数目


# 行列交叉比对，寻找成对的末端序列
for i in range(layout_id.shape[0]):
    print('Reading the %s' %i)
    for j in range(layout_id.shape[1]):
        query = j
        target = i
        BAC_id = layout_id[i,j]
        os.system('python /home/wxu/software/toolkits/find_overlap_sequence_by_blat_20181112.py -q ../3_trimmed_pool/Y%s_BES.fasta -t ../3_trimmed_pool/X%s_BES.fasta -o BES_%s.fasta' % (query +1, target +1, BAC_id.decode()))

        #read  BES to dict
        if os.path.exists(str('BES_' + BAC_id.decode() + '.fasta')) == False:
            os.system('touch BES_%s.fasta' % (BAC_id.decode()))
        BES_library_0[BAC_id] = [ i for i in FastaReader(str('BES_' + BAC_id.decode() + '.fasta'))]



# except false passitive
def difference_set():
#for times in range(5):                       # run five rounds
    for i in range(layout_id.shape[0]):
        print('Reading the %s' %i)
        for j in range(layout_id.shape[1]):
            this = layout_id[i, j]
            if len(BES_library_0[this]) > 3 :    # only discuss multi BESs, the only one BES need to skip

                X_ids_except_this = [x for x in layout_id[i, :] if x !=  this]
                Y_ids_except_this = [y for y in layout_id[:, j] if y != this]

                X_contigs_except_this = [k.name for k in flatten([BES_library_0[i][1::2] for i in X_ids_except_this])]
                Y_contigs_except_this = [k.name for k in flatten([BES_library_0[i][::2] for i in Y_ids_except_this])]

                del_index = []
                for m in  range(int(len(BES_library_0[this])/2)):
                    if BES_library_0[this][m*2 +1].name in X_contigs_except_this and BES_library_0[this][m*2].name in Y_contigs_except_this :
                        del_index.append(m*2+1)
                        del_index.append(m*2)
                BES_library_0[this] = [v for index , v in enumerate(BES_library_0[this]) if index not in del_index]


            num_bes[i, j] = int(len(BES_library_0[this]) / 2)  # count the number of BESs
            with open('BES_'+ this.decode() + '_1.fasta', 'w') as handle:
                for k in BES_library_0[this]:
                    k.write_to_fasta_file(handle)
    np.save('the_number_of_BESs', num_bes)

#位点末端序列唯一性
def only():
    for i in range(layout_id.shape[0]):
        for j in range(layout_id.shape[1]):
            this = layout_id[i, j]
            if len(BES_library_0[this]) == 2 :   # a pair of BESs

                for row_id in [id for id in layout_id[i,:] if id != this]:
                    if len(BES_library_0[row_id]) > 2:
                        for x_index in range(len(BES_library_0[row_id])):
                            if x_index % 2 == 0:
                                continue
                            else:
                                print(i,j,this,row_id,x_index)
                                if BES_library_0[this][1].name == BES_library_0[row_id][x_index].name:
                                    del BES_library_0[row_id][x_index-1:x_index+1]
                                    break

                for column_id in [id for id in layout_id[:,j] if id != this]:
                    if len(BES_library_0[column_id]) > 2:
                        for y_index in range(len(BES_library_0[column_id])):
                            if y_index % 2 == 0:
                                if BES_library_0[this][0].name == BES_library_0[column_id][y_index].name:
                                    del BES_library_0[column_id][y_index:y_index+2]
                                    break


            num_bes[i, j] = int(len(BES_library_0[this]) / 2)  # count the number of BESs
            with open('BES_'+ this.decode() + '_1.fasta', 'w') as handle:
                for k in BES_library_0[this]:
                    k.write_to_fasta_file(handle)
    np.save('the_number_of_BESs', num_bes)

##chose a method to adjust BAC end sequence
only()
only()
difference_set()
only()

#筛选后成对合并
for i in range(layout_id.shape[0]):
    for j in range(layout_id.shape[1]):
        query = j
        target = i
        BAC_id = layout_id[i,j].decode()
        print(BAC_id)
        if os.path.exists('BES_%s_1.fasta' %(BAC_id)):
            clone_list = [k for k in FastaReader('BES_%s_1.fasta' %(BAC_id))]

            if len(clone_list) == 2 :
                seq = TwoScaffoldsAlignment(SeqA=clone_list[0].seq.decode(), SeqB=clone_list[1].seq.decode()).merge()
                HTSeq.Sequence(name=BAC_id, seq=seq.encode()).write_to_fasta_file(handle_file)
                HTSeq.Sequence(name=BAC_id, seq=seq.encode()).write_to_fasta_file(handle_file_valid)
            elif len(clone_list) > 2:  #若位点包含多条末端，在每条序列后面添加“_”和数字编号
                for l in range(int(len(clone_list)/2)):
                    seq = TwoScaffoldsAlignment(SeqA=clone_list[l*2].seq.decode(), SeqB=clone_list[l*2 + 1].seq.decode()).merge()
                    name = BAC_id + '_'+ str(l+1)
                    HTSeq.Sequence(name=name, seq=seq.encode()).write_to_fasta_file(handle_file)
                    HTSeq.Sequence(name=name, seq=seq.encode()).write_to_fasta_file(handle_file_invalid)

handle_file.close()
handle_file_valid.close()
handle_file_invalid.close()
```

### 4 BAC定位

#### 4.1 末端序列比对
```bash
blastn -db Panicum_miliaceum_longmi4.fna -query PM-B-BES.fasta -outfmt '6 qaccver saccver pident length mismatch gapopen qlen qstart qend slen sstart send evalue bitscore sstrand qcovhsp' -qcov_hsp_perc 99 -perc_identity 99 -lcase_masking > PM-B-BES_longmi4.unmasked.blast
```

#### 4.2 GFF3格式形成

```Python3
class BAC_Recorder:
    def __init__(self, record):
        self._record = record
        self._name = self._record.iloc[0, 16]
        self._Forward = pd.DataFrame(columns=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17])
        self._Reverse = pd.DataFrame(columns=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17])
        self._gff = pd.DataFrame(columns=['chr','source','type','start','end','score','strand','phase','attributes'])


        self.split()
        self._Forward = self.merge(BES=self._Forward)
        self._Reverse = self.merge(BES=self._Reverse)
        self.conbinates()

    def split(self):
        for strand,BES_group in self._record.groupby([17]):
            BES_group = BES_group.sort_values(by=[13],ascending=False)
            if BES_group.iloc[0,17] == 'F':
                self._Forward = pd.concat([BES_group,self._Forward])
            elif BES_group.iloc[0,17] == 'R':
                self._Reverse = pd.concat([BES_group,self._Reverse])

    def merge(self, BES, edit=6):
        num_rows = BES.shape[0]
        BES[18] = -1
        if num_rows :
            for first in range(num_rows-1):
                for second in range(first+1,num_rows):
                    if BES.iloc[first,18] != -1 :
                        continue
                    else:
                        if BES.iloc[first,1] == BES.iloc[second,1] and \
                        (abs(BES.iloc[first,10] - BES.iloc[second,10]) < edit or \
                         abs(BES.iloc[first,11] - BES.iloc[second,11]) < edit ):
                            BES.iloc[first,18] = first
                            BES.iloc[second,18] = first
        return BES.drop_duplicates(subset=[18])

    def conbinates(self, max_length=200000):
        if self._Forward.shape[0] and self._Reverse.shape[0]:
            FLAG = False
            for F in range(self._Forward.shape[0]):
                for R in range(self._Reverse.shape[0]):
                    start = min(self._Forward.iloc[F, 10], self._Forward.iloc[F, 11], \
                                self._Reverse.iloc[R, 10], self._Reverse.iloc[R, 11])
                    end   = max(self._Forward.iloc[F, 10], self._Forward.iloc[F, 11], \
                                self._Reverse.iloc[R, 10], self._Reverse.iloc[R, 11])
                    if self._Forward.iloc[F, 1] == self._Reverse.iloc[R, 1] and \
                    self._Forward.iloc[F, 14] != self._Reverse.iloc[R, 14]  and \
                    end - start < max_length:
                        self._gff.loc[self._name] = [self._Forward.iat[F,1],'GResource','BES', str(start), str(end), \
                                             '.','.','.',str('ID=' + self._name + ';Name=' + self._name)]
                        self._gff.loc[str(self._name + '-F')] = [self._Forward.iat[F, 1], 'GResource', 'BES_end', \
                                    min(self._Forward.iat[F, 10], self._Forward.iat[F, 11]), \
                                    max(self._Forward.iat[F, 10], self._Forward.iat[F, 11]), \
                                    '.', self._Forward.iat[F, 14], '.', str('Parent=' + self._name)]
                        self._gff.loc[str(self._name + '-R')] = [self._Reverse.iat[R, 1], 'GResource', 'BES_end', \
                                    min(self._Reverse.iat[R, 10], self._Reverse.iat[R, 11]), \
                                    max(self._Reverse.iat[R, 10], self._Reverse.iat[R, 11]), \
                                    '.', self._Reverse.iat[R, 14], '.', str('Parent=' + self._name)]
                    elif self._Forward.iloc[F, 18] != -1:
                        self._gff.loc[self._name] = [self._Forward.iat[F, 1], 'GResource', 'BES_end', \
                                             min(self._Forward.iat[F, 10], self._Forward.iat[F, 11]), \
                                             max(self._Forward.iat[F, 10], self._Forward.iat[F, 11]), \
                                             '.', self._Forward.iat[F, 14], '.', str('ID=' + self._name + ';Name=' + self._name)]
                        FLAG = True
                        break
                    elif self._Reverse.iloc[R, 18] != -1:
                        self._gff.loc[self._name] = [self._Reverse.iat[R, 1], 'GResource', 'BES_end', \
                                             min(self._Reverse.iat[R, 10], self._Reverse.iat[R, 11]), \
                                             max(self._Reverse.iat[R, 10], self._Reverse.iat[R, 11]), \
                                             '.', self._Reverse.iat[R, 14], '.', str('ID=' + self._name + ';Name=' + self._name)]
                        FLAG = True
                        break
                if FLAG:
                    break


        elif self._Forward.shape[0]: # Only forward is recorded
            for F in range(self._Forward.shape[0]):
                self._gff.loc[self._name] = [self._Forward.iat[F, 1], 'GResource', 'BES_end', \
                                             min(self._Forward.iat[F, 10], self._Forward.iat[F, 11]), \
                                             max(self._Forward.iat[F, 10], self._Forward.iat[F, 11]), \
                                             '.', self._Forward.iat[F, 14], '.', str('ID=' + self._name + ';Name=' + self._name)]

        elif self._Reverse.shape[0]:   # Only reverse BES is recorded
            for R in range(self._Reverse.shape[0]):
                self._gff.loc[self._name] = [self._Reverse.iat[R, 1], 'GResource', 'BES_end', \
                                             min(self._Reverse.iat[R, 10], self._Reverse.iat[R, 11]), \
                                             max(self._Reverse.iat[R, 10], self._Reverse.iat[R, 11]), \
                                             '.', self._Reverse.iat[R, 14], '.', str('ID=' + self._name + ';Name=' + self._name)]




import pandas as pd
import collections


def main(blast):
    #add two field "name" and "end" into 17th and 18th respectively
    for i in range(blast.shape[0]):
        blast.loc[i, 16] = blast.loc[i, 0].split('_')[0]
        blast.loc[i, 17] = blast.loc[i, 0].split('_')[-1]
    blast[:3]

    # create a new DataFrame with 9 columns of gff3 format
    gff = pd.DataFrame(columns=['chr','source','type','start','end','score','strand','phase','attributes'])

    # group by BAC name
    for BES_name, BES_groups in blast.groupby(16):
        BAC = BAC_Recorder(BES_groups)
        gff = gff.append(BAC._gff)

    return gff
if __name__ == '__main__':
    blast = pd.read_table('PM-B-BES_short_and_long.blast',header=None)
    GFF3 = main(blast=blast)
    GFF3.to_csv('PM-B-BES_20190407.gff3',index=None,sep='\t')
```